# Data Download

In [3]:
import cdsapi
from tqdm import tqdm
from loguru import logger


years = range(1940, 2025)

logger.add("1.0-iragca-cds-data.log", rotation="500 MB", retention="7 days", compression="zip")
logger.info(f"Starting download of ERA5 data from {years[0]} to {years[-1]}.")
for year in tqdm(years, desc="Downloading data", unit="year"):
    try:
        dataset = "derived-era5-single-levels-daily-statistics"
        request = {
            "product_type": "reanalysis",
            "variable": ["2m_temperature"],
            "year": f"{year}",
            "month": [
                "01", "02", "03",
                "04", "05", "06",
                "07", "08", "09",
                "10", "11", "12"
            ],
            "day": [
                "01", "02", "03",
                "04", "05", "06",
                "07", "08", "09",
                "10", "11", "12",
                "13", "14", "15",
                "16", "17", "18",
                "19", "20", "21",
                "22", "23", "24",
                "25", "26", "27",
                "28", "29", "30",
                "31"
            ],
            "daily_statistic": "daily_mean",
            "time_zone": "utc+08:00",
            "frequency": "1_hourly"
        }

        client = cdsapi.Client()
        client.retrieve(dataset, request).download()
    except Exception as e:
        logger.error(f"Error downloading data for year {year}. Error: {e}")
        continue
    else:
        logger.info(f"Downloaded data for year {year}.")


2025-02-05 19:57:28.654 | INFO     | __main__:<module>:9 - Starting download of ERA5 data from 1940 to 2024.


2025-02-05 19:57:29,496 WARNING [2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using the correct syntax for your API request.
2025-02-05 19:57:30,658 INFO [2025-01-09T00:00:00] Please be aware that ERA5 data from 1st January 2025 was degraded and is being corrected. Watch the [Forum announcement](https://forum.ecmwf.int/t/era5-data-from-1st-january-2025-was-degraded-and-is-being-corrected/10689) for updates.
2025-02-05 19:57:30,659 INFO Request ID is 6b0a324b-7b28-4ac5-b937-a293950d12e7
2025-02-05 19:57:30,948 INFO status has been updated to accepted
2025-02-05 19:57:36,520 INFO status has been updated to running
